In [ ]:
#Written lihua chen,  lihua.chen@mse.gatech.edu
import numpy as np
import pandas as pd
import os
from pymatgen import Structure
import pandas as pd
from pymatgen.analysis.local_env import VoronoiNN
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers.structure import (DensityFeatures,SiteStatsFingerprint,GlobalSymmetryFeatures, StructuralHeterogeneity, StructureComposition,PartialRadialDistributionFunction)
from matminer.featurizers.composition import *
from matminer.featurizers.conversions import StrToComposition,CompositionToOxidComposition
from matminer.featurizers.site import *
from matminer.featurizers.function import FunctionFeaturizer

path=''
df = pd.read_csv('expt-inorganic-k-test.csv')
df['structure'] = 'default value'

ID=df.ID
n=(len(df.ID))
for i in range(n):
    structures = Structure.from_file(path+'structure/'+str(df.ID[i])+'.cif')
    df['structure'][i]=structures

## Apply a composition featurizer
df = StrToComposition().featurize_dataframe(df, "formular")

epdescriptors = ["X", "row", "group", "block", "atomic_mass", "atomic_radius", "mendeleev_no", "velocity_of_sound",
                        "thermal_conductivity", "melting_point",  "bulk_modulus",
                        "coefficient_of_linear_thermal_expansion"]
magpifeatures=["GSvolume_pa","CovalentRadius"]
demlfeatures = ["atom_num",  "heat_fusion","molar_vol",
                        "boiling_point", "heat_cap",
                        "first_ioniz"]
stats = ["mean","minimum", "maximum"]
#statsbondlength=['holder_mean::%d' % d for d in range(-4, 4 + 1)]
                                                 
featurizer = MultipleFeaturizer([
    SiteStatsFingerprint(CoordinationNumber(nn=VoronoiNN(weight='area'),use_weights="effective"),stats=stats),
#    SiteStatsFingerprint(GeneralizedRadialDistributionFunction.from_preset('gaussian')),
#    SiteStatsFingerprint(AngularFourierSeries.from_preset('gaussian')),
    SiteStatsFingerprint(AverageBondLength(VoronoiNN()),stats=stats),
    SiteStatsFingerprint(AverageBondAngle(VoronoiNN()),stats=stats),
#    SiteStatsFingerprint(AGNIFingerprints()),
    DensityFeatures(),
    GlobalSymmetryFeatures("spacegroup_num"),
    StructureComposition(ValenceOrbital(props=['avg'])),
    StructureComposition(ElementProperty(data_source = "pymatgen",features=epdescriptors, stats=stats)),
    StructureComposition(ElementProperty(data_source = "magpie",features=magpifeatures, stats=stats)),
    StructureComposition(ElementProperty(data_source = "deml",features=demlfeatures, stats=stats))                         
#    StructureComposition(ElectronAffinity()),
])                                                   


                                                 

df = featurizer.featurize_dataframe(df,'structure')

print("There are {} possible descriptors:\n\n{}".format(df.shape[1], df.columns.values))